# Swin Transformer - base and adversarial configurations
This notebook runs base (non-adversarial) and AutoAttack versions of the Swin Transformer on two datasets: the NIH Chest X-ray classification dataset and the ISIC Skin Cancer classification dataset.

In [1]:
# Installations & mount Drive
!pip install timm==0.4.12 yacs==0.1.8 adversarial-robustness-toolbox
!pip install -U PyYAML
!pip install git+https://github.com/fra31/auto-attack
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 376 kB 13.0 MB/s 
     |████████████████████████████████| 1.3 MB 64.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 14.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-pkgzb79b
  Running command git clone -q https://github.com/fra31/auto-attack /tmp/pip-req-build-pkgzb79b
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=34538 sha256=bba56b71a37d449fb38eb048b508bdefe5b51b33ae43f3b65e5fafc38282a79a
  Stored in directory: /tmp/pip-ephem-wheel-cache-

In [2]:
# remove hidden ipynb checkpoints and .DS_Store from data folders to avoid later FileNotFoundError
!rm -R /content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic/isic_org/.ipynb_checkpoints
!rm -R /content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic/isic_org/.DS_Store

rm: cannot remove '/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic/isic_org/.ipynb_checkpoints': No such file or directory
rm: cannot remove '/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic/isic_org/.DS_Store': No such file or directory


Run Swin Transformer configurations

NIH Chest X-ray dataset

In [ ]:
# Run base swin transformer on clean chest x-ray dataset
%%bash
#python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/main.py --cfg /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml --resume /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/swin_tiny_patch4_window7_224.pth --data-path /content/drive/MyDrive/MLSP_Masters/ECE_697/data/imagenet --batch-size 128 --output /content/drive/MyDrive/MLSP_Masters/ECE_697/data/swin_chxray_base_output --use-checkpoint
# Evaluate base swin transformer on noisy chest x-ray dataset (after 40 epochs of clean training)
python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/main.py --cfg /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml --resume /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/ckpt_epoch_150.pth --data-path /content/drive/MyDrive/MLSP_Masters/ECE_697/data/imagenet --batch-size 128 --output /content/drive/MyDrive/MLSP_Masters/ECE_697/data/swin_chxray_base_output --use-checkpoint


In [ ]:
# Run adversarial swin transformer on chest x-ray dataset
!pip install git+https://github.com/fra31/auto-attack
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345 /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/swin_autoattack.py --cfg=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml --data_dir=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/swin_auto_1000 --csv=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/swin_autoattack_1000.csv --model=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/ckpt_epoch_150.pth --save_dir=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/swin_chxray_adv_outputs/results --batch_size=128 --num_workers=1 --log_path=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/swin_chxray_adv_outputs 
#!python -m torch.distributed.launch --nproc_per_node 8 --master_port 12345 ~/Models/Swin-Transformer/swin_autoattack.py --cfg=configs/swin/swin_tiny_patch4_window7_224.yaml --data_dir=~/Data/chxray/raw_imgs --data_path=~/Data/chxray/imagenet --csv=~/Data/chxray/autoattack_labels.csv --model=ckpt_epoch_150.pth --save_dir=~/Models/swin_chxray_adv_output/results --batch_size=128 --num_workers=1 --log_path=~/Models/swin_chxray_adv_outputs/results/logger.txt

ISIC Dataset

In [ ]:
# Run base swin transformer on clean ISIC dataset
#!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/main.py --cfg /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml --resume /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/swin_tiny_patch4_window7_224.pth --data-path /content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic/isic_org --batch-size 128 --opts TRAIN.EPOCHS 200 TRAIN.WARMUP_EPOCHS 10 TRAIN.BASE_LR 0.0002 --output /content/drive/MyDrive/MLSP_Masters/ECE_697/data/swin_isic_base_output #--use-checkpoint
# Evaluate base swin transformer on noisy ISIC dataset (after 40 epochs of clean training)
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/main.py --cfg /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml --resume /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/isic_ckpt_epoch_40.pth --data-path /content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic/isic_org --batch-size 128 --opts TRAIN.EPOCHS 200 TRAIN.WARMUP_EPOCHS 10 TRAIN.BASE_LR 0.0002 --output /content/drive/MyDrive/MLSP_Masters/ECE_697/data/swin_isic_base_output #--use-checkpoint


Streaming output truncated to the last 5000 lines.
          [-7.0882e-01,  1.8339e-01,  1.3539e-01,  ...,  1.0579e-02,
           -2.3561e+00,  7.0997e-01]],

         [[-1.3196e-02, -5.6456e-01,  2.9099e-01,  ...,  1.6794e-01,
            1.8828e-01, -3.6983e-01],
          [ 4.5861e-01,  9.0263e-01,  5.6270e-01,  ..., -1.4510e+00,
            1.1523e+00,  7.0928e-02],
          [ 1.7833e+00, -3.0255e-01, -7.0516e-02,  ...,  9.4050e-01,
            1.3395e+00, -5.3006e-01],
          ...,
          [-7.6399e-01,  1.4480e+00, -2.4176e+00,  ...,  6.0753e-01,
            6.5063e-03,  4.1400e-01],
          [-2.5970e-02, -3.8886e-01, -1.1551e+00,  ..., -3.7622e-01,
           -9.9808e-01, -2.0947e-01],
          [-1.5266e-01,  1.6412e-01, -4.5688e-01,  ..., -1.2599e+00,
           -2.5127e-01, -2.6180e+00]]],


        [[[ 4.9167e-01, -1.0794e+00,  2.1938e-01,  ...,  6.5393e-01,
           -1.1635e+00,  8.5410e-01],
          [ 6.1913e-03,  2.0606e+00, -2.7262e+00,  ..., -1.0696e-01,
   

In [ ]:
while True:
  print(1)

Streaming output truncated to the last 5000 lines.
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

KeyboardInterrupt: ignored

In [ ]:
# Run adversarial swin transformer on ISIC dataset

!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345 /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/swin_autoattack.py --cfg=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml --data_dir=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_1000 --csv=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_1000.csv --model=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/isic_ckpt_epoch_150.pth --save_dir=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/swin_isic_adv_outputs/results --batch_size=224 --num_workers=1 --log_path=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/swin_isic_adv_outputs --csv_tr=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_train.csv --csv_ts=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_test.csv


/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Tutel has not been installed. To use Swin-MoE, please install Tutel; otherwise, just ignore this.
Namespace(accumulation_steps=None, amp_opt_level=None, batch_size=224, cache_mode='part', cfg='/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml', csv='/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_1000.csv', csv_tr='/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_train.csv', csv_ts='/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto

In [ ]:
!python /content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/swin_autoattack.py --cfg=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml --data_dir=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_1000 --csv=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_1000.csv --model=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/isic_ckpt_epoch_150.pth --save_dir=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/swin_isic_adv_outputs/results --batch_size=224 --num_workers=1 --log_path=/content/drive/MyDrive/MLSP_Masters/ECE_697/models/swin_isic_adv_outputs --csv_tr=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_train.csv --csv_ts=/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_test.csv

Tutel has not been installed. To use Swin-MoE, please install Tutel; otherwise, just ignore this.
Namespace(accumulation_steps=None, amp_opt_level=None, batch_size=224, cache_mode='part', cfg='/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/configs/swin/swin_tiny_patch4_window7_224.yaml', csv='/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_1000.csv', csv_tr='/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_train.csv', csv_ts='/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_test.csv', data_dir='/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic_auto_1000', data_path='./data', disable_amp=False, epsilon=0.03137254901960784, eval=False, individual=False, local_rank=0, log_path='/content/drive/MyDrive/MLSP_Masters/ECE_697/models/swin_isic_adv_outputs', model='/content/drive/MyDrive/MLSP_Masters/ECE_697/models/Swin-Transformer/isic_ckpt_epoch_150.pth', n_ex=1000, norm='Linf', num_workers=1, opts=None, output='output', pretrained=None, 